<a href="https://colab.research.google.com/github/amritanshu009/DSAexperiments/blob/main/RoBert_less_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Reddit_Data.csv to Reddit_Data.csv


In [ ]:
import pandas as pd
import numpy as np
import torch
import os

from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

In [ ]:
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

# ✅ Disable WandB logs in Colab
os.environ["WANDB_DISABLED"] = "true"

# ✅ Load and clean dataset from Colab path
df = pd.read_csv("/content/Reddit_Data.csv")
df = df[['clean_comment', 'category']].dropna()

# ✅ Reduce dataset size for faster training (optional)
df = df.sample(3000, random_state=42).reset_index(drop=True)

# ✅ Classic ML Model with Logistic Regression
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42)
vectorizer = TfidfVectorizer(max_features=3000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

logistic_model = LogisticRegression()
logistic_model.fit(X_train_vec, y_train)

# ✅ Evaluation of Logistic Regression
y_pred = logistic_model.predict(X_test_vec)
print("\nLogistic Regression Performance:")
print(classification_report(y_test, y_pred))

# ✅ Inference using Logistic Regression
def predict_sentiment_logistic(text):
    vec = vectorizer.transform([text])
    return logistic_model.predict(vec)[0]

print("Predicted Sentiment (Logistic):", predict_sentiment_logistic("Buddhism is very peaceful and calming"))

# ✅ Label Mapping for RoBERTa
label_map = {-1: 0, 0: 1, 1: 2}
reverse_map = {0: -1, 1: 0, 2: 1}
df['label'] = df['category'].map(label_map)

# ✅ Convert to HuggingFace Dataset
hf_dataset = Dataset.from_pandas(df[['clean_comment', 'label']])
hf_dataset = hf_dataset.train_test_split(test_size=0.2)

# ✅ Tokenization for RoBERTa
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize(example):
    return tokenizer(example["clean_comment"], truncation=True, padding="max_length", max_length=64)

tokenized_dataset = hf_dataset.map(tokenize, batched=True)

# ✅ Load RoBERTa model and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
model.to(device)

# ✅ Metrics for Trainer
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# ✅ Training Arguments
training_args = TrainingArguments(
    output_dir="./roberta_results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs"
)

# ✅ Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# ✅ Train the RoBERTa model
trainer.train()

# ✅ RoBERTa Inference Function
def predict_sentiment_roberta(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return reverse_map[predicted_class]

# ✅ Test RoBERTa Prediction
sample = "I love the peaceful teachings of Buddhism."
print("Predicted Sentiment (RoBERTa):", predict_sentiment_roberta(sample))


Logistic Regression Performance:
              precision    recall  f1-score   support

          -1       0.65      0.23      0.34       129
           0       0.69      0.82      0.75       222
           1       0.69      0.80      0.74       249

    accuracy                           0.69       600
   macro avg       0.68      0.62      0.61       600
weighted avg       0.68      0.69      0.66       600

Predicted Sentiment (Logistic): 1


Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-7-f22b459ed5f9>:80: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


Predicted Sentiment (RoBERTa): 1


In [ ]:
metrics = trainer.evaluate()
print("\nRoBERTa Evaluation Metrics:")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")


RoBERTa Evaluation Metrics:
eval_loss: 0.8015
eval_accuracy: 0.6517
eval_f1: 0.5700
eval_precision: 0.5169
eval_recall: 0.6517
eval_runtime: 133.6990
eval_samples_per_second: 4.4880
eval_steps_per_second: 0.5610
epoch: 1.0000
